#Setup google-drive mounting (optional)

In [ ]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')

In [ ]:
# # prompt: create a symbolic link to a google drive workdir 'xyz' to the root of colab

# # Specify the path to your folder
# gdrive = '/content/drive/MyDrive/AI/2025'
# workdir = '/datasets'
# slink = '/content' + workdir
# fullpath = gdrive + workdir

# # Check if the folder exists
# if os.path.exists(fullpath):
#   # Create the symbolic link
#   try:
#     os.symlink(fullpath, slink)
#     print(f"Symbolic link created from '{fullpath}' to '{slink}'")
#   except FileExistsError:
#     print(f"Symbolic link '{slink}' already exists.")
#   except OSError as e:
#     print(f"Error creating symbolic link: {e}")
# else:
#   print(f"Error: Folder '{fullpath}' not found.")

# # !ls -lh /content

In [ ]:
# !unzip datasets/datasets.zip -d ./
# !rm datasets/datasets.zip

#Loading real test dataset

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

subset = '/crypto/hourly/'
folder = './datasets' + subset

IB = 100.0 # initial balance for each asset
SPLIT_DATE = '2024-08-26'

In [ ]:
df = pd.read_csv(folder + 'dataset.csv')
df

In [ ]:
date_column_name = df.columns[0]
date_format = '%Y-%m-%d' if date_column_name.lower() == 'date' else '%Y-%m-%d %H:%M:%S'

date_column_name, date_format

In [ ]:
# Converter a coluna para datetime removendo o fuso horário
if date_column_name.lower() == 'date':
    df[date_column_name] = pd.to_datetime(df[date_column_name])
else:
    df[date_column_name] = pd.to_datetime(df[date_column_name]).dt.tz_convert(None)

df

In [ ]:
split_date = pd.to_datetime(SPLIT_DATE)

# Find the first row where the date is equal or greater than split_date
def get_split_date_index(df, split_date):
  for i in range(len(df)):
    if df.iloc[i, 0] >= split_date:
      return i

split_idx = get_split_date_index(df, split_date)
split_idx, df.iloc[split_idx, 0]

In [ ]:
split_idx += 17 # adjusting for other forecasts

In [ ]:
df_test = df.iloc[split_idx-1:, :].drop('Real', axis=1)
df_test

#Loading saved backtests for analysis

In [ ]:
# prompt: load all saved backtests from folder into backtests list
import os

def load_backtests(path):
  # Assuming 'backtests' is the directory containing the CSV files
  backtest_dir = os.path.join(path, 'backtests')
  backtests = []
  titles = []
  i = 0

  # Iterate through the files in the directory
  for filename in os.listdir(backtest_dir):
    if filename.endswith(".csv"):
      filepath = os.path.join(backtest_dir, filename)
      try:
        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(filepath)
        backtests.append(df)
        titles.append(filename.removesuffix('.csv'))
        print(f"Backtest loaded[{i}]: {filename}")
        i += 1
      except pd.errors.EmptyDataError:
        print(f"Warning: Skipping empty file: {filename}")
      except pd.errors.ParserError:
        print(f"Warning: Skipping file with parsing error: {filename}")

  return backtests, titles

In [ ]:
# loading backtests without cross-validation
backtests, titles = load_backtests(folder)

In [ ]:
# loading backtests with plain cross-validation (no refit)
cross_plain_path = folder + 'cross-validation/plain/' # forecasts with cross-validation (no refit)
# cross_refit_path # folder + 'cross-validation/refit/' # forecasts with cross-validation and refit

plain_backtests, plain_titles = load_backtests(cross_plain_path)
print(plain_titles)

In [ ]:
#cross_plain_path = folder + 'cross-validation/plain/' # forecasts with cross-validation (no refit)
cross_plain_btc_path = folder + 'cross-validation/plain-btc/' # forecasts with cross-validation (no refit)
# cross_refit_path # folder + 'cross-validation/refit/' # forecasts with cross-validation and refit

plain_btc_backtests, plain_btc_titles = load_backtests(cross_plain_btc_path)
print(plain_btc_titles)

In [ ]:
# cross_plain_path = folder + 'cross-validation/plain/' # forecasts with cross-validation (no refit)
# cross_plain_btc_path = folder + 'cross-validation/plain-btc/' # forecasts with cross-validation (no refit)
cross_refit_path = folder + 'cross-validation/refit/' # forecasts with cross-validation and refit

refit_backtests, refit_titles = load_backtests(cross_refit_path)
print(plain_btc_titles)

In [ ]:
# folder # './datasets/crypto/hourly/'
# path # folder + 'forecasts/' # forecasts without cross-validation
# cross_plain_path # folder + 'cross-validation/plain/' # forecasts with cross-validation (no refit)
# cross_refit_path # folder + 'cross-validation/refit/' # forecasts with cross-validation and refit

# df # original dataset loaded from .csv

# date_column_name # 'Datetime' or 'Date'
# date_format # '%Y-%m-%d' or '%Y-%m-%d %H:%M:%S'
# split_idx # 11616 (index where training and testing datasets are split, separated by provided date)

# df_test # df from split_idx dropped by column 'Real'

# MODEL_NAMES # list of all used models (appended by 'dumb_4cast' later)

# forecasts # forecasts without cross-validation
# plain_forecasts # forecasts with cross-validation (no refit)
# refit_forecasts # forecasts with cross-validation and refit

# IB # 100.0 initial balance for each asset
# SPLIT_DATE = '2024-08-26'

# bh_df # baseline buy-and-hold dataframe
# dumb_4cast # simple one day shit from df_train

# backtests # backtests calculated from forecasts
# plain_backtests # backtests calculated from plain_forecasts
# refit_backtests # backtests calculated from refit_forecasts

In [ ]:
backtests[0]

In [ ]:
# prompt: create a dataframe btsums with same indexes of wallet[0] and one column for each entry in wallets

# Assuming 'backtests' list is already populated as in the provided code
# and 'wallet0' is correctly assigned to backtests[0].

btsums = pd.DataFrame(index=backtests[0].index)  # Use wallet0's index
btsums[date_column_name] = backtests[0][date_column_name]
cols = len(backtests[0].columns) - 1

for i, df in enumerate(backtests):
    # Calculate the sum of all columns except the first one (date)
    column_sum = df.iloc[:, 1:].sum(axis=1)
    btsums[titles[i]] = column_sum / cols
btsums

In [ ]:
# prompt: plot btsums in one line graph

import plotly.graph_objects as go

fig = go.Figure()

for title in titles:
    fig.add_trace(go.Scatter(x=btsums[date_column_name], y=btsums[title], mode='lines', name=title))

fig.update_layout(title='BTSums One Line Graph',
                  xaxis_title='Date',
                  yaxis_title='Value')
fig.show()

In [ ]:
# prompt: create a function to plot with plotly each dataset from backtests[], all columns of each one

import plotly.graph_objects as go

def plot_backtests(backtests, titles):
  """Plots each dataset from backtests[] using Plotly.

  Args:
      backtests: A list of pandas DataFrames.
  """

  for i, df in enumerate(backtests):
    fig = go.Figure()
    for column in df.columns[1:]:  # Exclude the date column
      fig.add_trace(go.Scatter(x=df[df.columns[0]], y=df[column], mode='lines', name=column))

    fig.update_layout(title=f"{titles[i].upper()}", xaxis_title="Date", yaxis_title="Value")
    fig.show()

plot_backtests(backtests, titles)

In [ ]:
# prompt: copy last row of wallet0 with title, sort it ascending, subtract each value from IB
def sort_last_row(df, initial_balance_to_subtract):
  last_row = df.tail(1).copy()
  last_row_subtracted = last_row.iloc[:, 1:] - initial_balance_to_subtract
  last_row_subtracted_t = last_row_subtracted.T
  last_row_subtracted_sorted = last_row_subtracted_t.sort_values(by=last_row_subtracted_t.columns[0], ascending=True)
  return last_row_subtracted_sorted.T

In [ ]:
# prompt: plot wallet0 in a bar graph using matplotlib, negative values on red, positive values on blue, create code as compact as possible

import matplotlib.pyplot as plt

# Assuming wallet0 is a pandas DataFrame with column names as asset names
# and the last row contains the final wallet values.

def plot_bar(wallet, title):
  # Extract the last row and remove the first column ('Date')
  last_row = wallet.iloc[-1, 1:]

  # Create a bar plot
  plt.figure(figsize=(15, 5))
  bars = plt.bar(last_row.index, last_row.values)

  # Set colors based on positive or negative values
  for bar, value in zip(bars, last_row.values):
    if value < 0:
      bar.set_color('red')
    else:
      bar.set_color('blue')

  plt.xlabel('Assets')
  plt.ylabel('Value')
  plt.title(title)
  plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
  plt.tight_layout() # Adjust layout to prevent labels from overlapping
  plt.show()

In [ ]:
for i in range(len(backtests)):
  wallet = sort_last_row(backtests[i], IB)
  display(wallet)
  plot_bar(wallet, titles[i].upper())

In [ ]:
# prompt: create a plotly animated bar graph over time from wallet, 30 frames per second

import plotly.graph_objects as go

# Assuming 'wallet0' DataFrame is already defined and contains a 'Date' column
# and columns for each asset.

idx = 5
wallet_sorted = sort_last_row(backtests[idx], IB)
wallet = backtests[idx][wallet_sorted.columns]

# Prepare the data for animation
frames = []
for i in range(len(wallet)):
    frame = go.Frame(
        data=[go.Bar(x=wallet.columns[1:], y=wallet.iloc[i, 1:])],
        name=str(i)  # Name each frame
    )
    frames.append(frame)

# Create the initial figure
fig = go.Figure(
    data=[go.Bar(x=wallet.columns[1:], y=wallet.iloc[0, 1:])],
    layout=go.Layout(
        title="Wallet Animation",
        xaxis_title="Assets",
        yaxis_title="Value",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 1, "redraw": True},
                                       "fromcurrent": True, "transition": {"duration": 0}}])])]
    ),
    frames=frames
)

# Show the animated figure
fig.show()

In [ ]:
!pip install bar_chart_race

In [ ]:
import bar_chart_race as bcr

idx = 3
wallet = backtests[idx]
#bcr.bar_chart_race(df=wallet.iloc[:,1:], filename='bar_chart_race_crypto_deepnpts.mp4')
bcr.bar_chart_race(
    df=wallet.iloc[:,1:],
    filename='bcr' + subset.replace('/', '_') + titles[idx] + '.mp4',
    title=titles[idx].upper() + ' Performance on ' + subset.replace('/', ' ').upper()  + ' Dataset',
    period_length=50,         # 1500 ms / 50 = 30x faster
    steps_per_period=10,      # fewer steps to keep pace smooth but fast
    interpolate_period=True
)

In [ ]:
import IPython
IPython.display.Audio("file_example_MP3_1MG.mp3", autoplay=True)